In [0]:
username = 'scook'
from IPython.display import display, HTML, clear_output
try:
    %reload_ext autotime
except:
    %pip install -U ipython-autotime ipywidgets codetiming openpyxl numpy pandas geopandas flaml[automl] git+https://github.com/AnotherSamWilson/miceforest.git
    dbutils.library.restartPython()
    clear_output()
    dbutils.notebook.exit('Rerun to use newly installed/updated packages')

import pathlib, shutil, warnings, requests, dataclasses, codetiming, numpy as np, pandas as pd, geopandas as gpd, flaml as fl, miceforest as mf
clear_output()
seed = 42
catalog = 'dev.bronze.'
root = pathlib.Path(f'/Workspace/Users/{username}@tarleton.edu/admitted_matriculation_predictor_2025/')
data = root/'data'
flags_raw = pathlib.Path('/Volumes/aiml/scook/scook_files/admitted_flags_raw')
flags_prc = pathlib.Path('/Volumes/aiml/flags/flags_volume/')

##########################################
############ helper functions ############
##########################################
tab = '    '
divider = '\n##############################################################################################################\n'

def listify(*args):
    """ensure it is a list"""
    if len(args)==1:
        if args[0] is None or args[0] is np.nan or args[0] is pd.NA:
            return list()
        elif isinstance(args[0], str):
            return [args[0]]
    try:
        return list(*args)
    except Exception as e:
        return list(args)

def setify(*args):
    """ensure it is a set"""
    return set(listify(*args))

def rjust(x, width, fillchar=' '):
    return str(x).rjust(width,str(fillchar))

def ljust(x, width, fillchar=' '):
    return str(x).ljust(width,str(fillchar))

def join(lst, sep='\n,', pre='', post=''):
    """flexible way to join list of strings into a single string"""
    return f"{pre}{str(sep).join(map(str,listify(lst)))}{post}"

def alias(dct):
    """convert dict of original column name:new column name into list"""
    return [f'{k} as {v}' for k,v in dct.items()]

def indent(x, lev=1):
    return x.replace('\n','\n'+tab*lev) if lev>0 else x

def subqry(qry, lev=1):
    """make qry into subquery"""
    return "(" + indent('\n'+qry.strip()+'\n)', lev)

def run(qry, show=False, sample='10 rows', seed=seed):
    """run qry and return dataframe"""
    L = qry.split(' ')
    if len(L) == 1:
        qry = f'select * from {catalog}{L[0]}'
        if sample is not None:
            qry += f' tablesample ({sample}) repeatable ({seed})'
    if show:
        print(qry)
    return spark.sql(qry).toPandas().prep()

def rm(path, root=False):
    path = pathlib.Path(path)
    if path.is_file():
        path.unlink()
    elif path.is_dir():
        if root:
            shutil.rmtree(path)
        else:
            for p in path.iterdir():
                rm(p, True)
    return path

############ pandas functions ############
pd.options.display.max_columns = None
def disp(df, rows=4, head=True, sort=False):
    """convenient display method"""
    with pd.option_context('display.min_rows', rows, 'display.max_rows', rows):
        print(df.shape)
        df = df.sort_index(axis=1) if sort else df
        missing = df.isnull().sum().to_frame().T
        X = pd.concat([df.dtypes.to_frame().T, missing, (missing/df.shape[0]*100).round(2), df.head(rows) if head else df.tails(rows)])
        display(HTML(X.to_html()))


def to_numeric(df, downcast='integer', errors='ignore', **kwargs):
    """convert to numeric dtypes if possible"""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return (
            df
            .apply(lambda s: s.astype('string').str.lower().str.strip() if s.dtype in ['object','string'] else s)  # prep strings
            .apply(lambda s: s if pd.api.types.is_datetime64_any_dtype(s) else pd.to_numeric(s, downcast=downcast, errors=errors, **kwargs))  # convert to numeric if possible
            .convert_dtypes()  # convert to new nullable dtypes
            .apply(lambda s: s.astype('Int64') if pd.api.types.is_integer_dtype(s) else s)
        )

def prep(df, **kwargs):
    h = lambda x: x.to_numeric(**kwargs).rename(columns=lambda s: s.lower().strip().replace(' ','_').replace('-','_') if isinstance(s, str) else s)
    idx = h(df[[]].reset_index())  # drop columns, reset_index to move index to columns, then apply g
    return h(df).reset_index(drop=True).set_index(pd.MultiIndex.from_frame(idx))  # set idx back to df's index

def wrap(fcn):
    """Make new methods work for Series and DataFrames"""
    def wrapper(X, *args, **kwargs):
        df = fcn(pd.DataFrame(X), *args, **kwargs)
        return None if df is None else df.squeeze() if isinstance(X, pd.Series) else df  # squeeze to series if input was series
    return wrapper

for f in [disp, to_numeric, prep]:
    """monkey-patch my helpers into Pandas Series & DataFrame classees so we can use df.method syntax"""
    setattr(pd.DataFrame, f.__name__, f)
    setattr(pd.Series, f.__name__, wrap(f))

#########################################
################## AMP ##################
#########################################
@dataclasses.dataclass
class Term():
    term_code: int = 202408
    cycle_day: int = None
    cycle_date: str = None
    overwrite: set = None

    #Allows self['attr'] and self.attr syntax
    def __contains__(self, key):
        return hasattr(self, key)
    def __getitem__(self, key):
        return getattr(self, key)
    def __setitem__(self, key, val):
        setattr(self, key, val)
    def __delitem__(self, key):
        if key in self:
            delattr(self, key)

    def __post_init__(self):
        self.term_code = int(self.term_code)
        self.overwrite = setify(self.overwrite)
        # this takes about 1 hour per campus so, to avoid accidental deletion requring lengthy re-creation, we force user to manual delete parquet files 
        self.overwrite.discard('drivetimes_s')
        self.overwrite.discard('drivetimes_m')
        self.overwrite.discard('drivetimes_w')
        self.overwrite.discard('drivetimes_r')
        self.get_terms()

    def get(self, fcn, dst, prereq=[], divide=True, read=True, **kwargs):
        nm = str(dst)
        if '/' in nm:
            dst = pathlib.Path(dst).with_suffix('.parquet')
            nm = dst.stem
        else:
            dst = data/f'{dst}/{self.term_code}/{dst}_{self.stem}.parquet'
        if nm in self.overwrite:
            del self[nm]
            dst.unlink(missing_ok=True)
            self.overwrite.remove(nm)
        new = False
        if not nm in self:
            if not dst.exists():
                new = True
                for f in listify(prereq):
                    f()
                print(f'creating {dst.name}: ', end='')
                with codetiming.Timer():
                    dst.parent.mkdir(parents=True, exist_ok=True)
                    df = pd.DataFrame(fcn(**kwargs)).prep()  # forced to wrap with explicit pd.DataFrame to due strange error under pandas 2.2.3 "Object of type PlanMetrics is not JSON serializable" with to_parquet
                    df.to_parquet(dst)
                if divide:
                    print(divider)
            if read:
                self[nm] = pd.read_parquet(dst)
            else:
                self[nm] = None
        return self[nm], new
##################################################
################# get drivetimes #################
##################################################
    def get_drivetimes(self, show=False):
        def fcn():
            print()
            campus_coords = {
                's': '-98.215784,32.216217',
                'm': '-97.432975,32.582436',
                # 'w': '-97.172176,31.587908',
                # 'r': '-96.467920,30.642055',
                }
            url = "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_zcta520_500k.zip"
            gdf = gpd.read_file(url).prep().set_index('zcta5ce20')  # get all ZCTA https://www.census.gov/programs-surveys/geography/guidance/geo-areas/zctas.html
            pts = gdf.sample_points(size=10, method="uniform").explode().apply(lambda g: f"{g.x},{g.y}")  # sample 10 random points in each ZCTA
            M = []
            for k, v in campus_coords.items():
                def inner_fcn():
                    print()
                    L = []
                    i = 0
                    di = 200
                    I = pts.shape[0]
                    while i < I:
                        u = join([v, *pts.iloc[i:i+di]],';')
                        url = f"http://router.project-osrm.org/table/v1/driving/{u}?sources={0}&annotations=duration,distance&fallback_speed=1&fallback_coordinate=snapped"
                        response = requests.get(url).json()
                        L.append(np.squeeze(response['durations'])[1:]/60)
                        i += di
                        print(k,i,round(i/I*100))
                    df = pts.to_frame()[[]]
                    df[k] = np.concatenate(L)
                    return df
                df, new = self.get(inner_fcn, root/f'drivetimes/drivetimes_{k}')
                M.append(df)
            # df = pd.concat(M, axis=1).groupby(level=0).min().stack().reset_index().set_axis(['zip','camp_code','drivetime'], axis=1)
            df = pd.concat(M, axis=1).groupby(level=0).min().stack().rename_axis(['zip','camp_code']).rename('drivetime')
            return df
        df, new = self.get(fcn, root/'drivetimes/drivetimes')
        return df

########################################################
################# get term information #################
########################################################
    def get_terms(self, show=False):
        def fcn():
            qry = f"""
select
    stvterm_code as term_code
    ,replace(stvterm_desc, ' ', '') as term_desc
    ,stvterm_start_date as start_date
    ,stvterm_end_date as end_date
    ,stvterm_fa_proc_yr as fa_proc_yr
    ,stvterm_housing_start_date as housing_start_date
    ,stvterm_housing_end_date as housing_end_date
    ,sobptrm_census_date as census_date
from
    {catalog}saturnstvterm as A
inner join
    {catalog}saturnsobptrm as B
on
    stvterm_code = sobptrm_term_code
where
    sobptrm_ptrm_code='1'
"""
            df = run(qry, show).set_index('term_code')
            df['stable_date'] = df['census_date'].apply(lambda x: x+pd.Timedelta(days=7+4-x.weekday())) # Friday of week following census
            return df
        df, new = self.get(fcn, data/'terms')
        self.cycle_day, self.cycle_date, self.stable_date, self.stem = self.get_cycle(self.term_code, self.cycle_day, self.cycle_date)
        return df


    def get_cycle(self, term_code, cycle_day=None, cycle_date=None, show=False):
        stable_date = self.terms.loc[int(term_code),'stable_date']
        if cycle_day is None:
            if cycle_date is None:
                cycle_date = pd.Timestamp.now()
            cycle_date = pd.to_datetime(cycle_date).normalize()
            cycle_day = (stable_date - cycle_date).days
        cycle_date = (stable_date - pd.Timedelta(days=cycle_day)).date()
        # cycle_date = str((stable_date - pd.Timedelta(days=cycle_day)).date())
        stem = f'{term_code}_{cycle_date}_{"-" if cycle_day < 0 else "+"}{rjust(abs(cycle_day),3,0)}'
        return cycle_day, cycle_date, stable_date, stem

#######################################################
############ process flags reports archive ############
#######################################################
    def get_spriden(self, show=False):
        # Get id-pidm crosswalk so we can replace id by pidm in flags below
        # GA's should not have permissions to run this because it can see pii
        if 'spriden' not in self:
            qry = f"""
            select distinct
                spriden_id as id,
                spriden_pidm as pidm
            from
                {catalog}saturnspriden as A
            where
                spriden_change_ind is null
                and spriden_activity_date between '2000-09-01' and '2025-09-01'
                and spriden_id REGEXP '^[0-9]+'
            """
            self.spriden = run(qry, show)
        return self.spriden


    def process_flags(self, show=False):
        # GA's should not have permissions to run this because it can see pii
        counter = 0
        divide = False
        for src in sorted(flags_raw.iterdir(), reverse=True):
            counter += 1
            if counter > 5:
                break
            a,b = src.name.lower().split('.')
            if b != 'xlsx' or 'melt' in a or 'admitted' not in a:
                print(a, 'SKIP')
                continue
            # Handles 2 naming conventions that were used at different times
            try:
                cycle_date = pd.to_datetime(a[:10].replace('_','-'))
                multi = True
            except:
                try:
                    cycle_date = pd.to_datetime(a[-6:])
                    multi = False
                except:
                    print(a, 'FAIL')
                    continue
            # print(a, dt.date())
            book = pd.ExcelFile(src, engine='openpyxl')
            # Again, handles the 2 different versions with different sheet names
            if multi:
                sheets = {sheet:sheet for sheet in book.sheet_names if sheet.isnumeric() and int(sheet) % 100 in [1,6,8]}
            else:
                sheets = {a[:6]: book.sheet_names[0]}
            for term_code, sheet in sheets.items():
                cycle_day, cycle_date, stable_date, stem = self.get_cycle(term_code, cycle_date=cycle_date)
                def fcn():
                    df = (
                        self.get_spriden()
                        .assign(cycle_day=cycle_day, cycle_date=cycle_date)
                        .merge(book.parse(sheet).prep(), on='id', how='right')
                        .drop(columns=['id','last_name','first_name','mi','pref_fname','street1','street2','primary_phone','call_em_all','email'], errors='ignore')
                    )
                    return df
                if self.get(fcn, flags_prc/f'{term_code}/flags_{stem}', read=False, divide=False)[1]:
                    divide = True
                    counter = 0
                    dst = flags_prc/f'flags_{term_code}.parquet'
                    dst.unlink(missing_ok=True)
        if divide:
            print(divider)
            self.combine_flags()


    def combine_flags(self, show=False):
        def fcn(term_code):
            F = sorted((flags_prc/f'{term_code}').glob('*.parquet'))+sorted((flags_prc/f'{term_code-2}').glob('*.parquet'))
            L = [pd.read_parquet(src) for src in F]
            df = pd.concat(L, ignore_index=True).prep()
            del L
            for k in ['dob',*df.filter(like='date').columns]:  # convert date columns
                if k in df:
                    df[k] = pd.to_datetime(df[k], errors='coerce')
            return df
        divide = False
        for x in flags_prc.iterdir():
            if x.is_dir():
                term_code = int(x.stem)
                if term_code%10==8:
                    if self.get(fcn, flags_prc/f'flags_{term_code}', read=False, divide=False, term_code=term_code)[1]:
                        divide = True
        if divide:
            print(divider)


    def get_flags(self, show=False):
        def fcn():
            df = (
                pd.read_parquet(flags_prc/f'flags_{self.term_code}.parquet')
                .query(f"cycle_date<='{self.cycle_date}'")
                .sort_values(['pidm','cycle_date'])
                .drop_duplicates(subset=['pidm','term_code'], keep='last')
            )
            df.loc[~df['state'].isin(self.states),'zip'] = pd.NA
            df['zip'] = df['zip'].str.split('-', expand=True)[0].str[:5]
            return df
        df, new = self.get(fcn, 'flags', self.combine_flags)
        return df

########################################################
############### get course registrations ###############
########################################################
    def get_registrations(self, show=False):
        def fcn():
            dct = {
                'sfrstcr_pidm':'pidm',
                'ssbsect_term_code':'term_code',
                'sgbstdn_levl_code':'levl_code',
                'sgbstdn_styp_code':'styp_code',
            }
            qry = f"""
select
    {indent(join(alias(dct)))}
    ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
    ,max(ssbsect_credit_hrs) as credit_hr
from
    {catalog}saturnsfrstcr as A
inner join
    {catalog}saturnssbsect as B
on
    sfrstcr_term_code = ssbsect_term_code
    and sfrstcr_crn = ssbsect_crn
inner join (
    select
        *
    from
        {catalog}sgbstdn_amp_v
    where
        sgbstdn_term_code_eff <= {self.term_code}
    qualify
        row_number() over (partition by sgbstdn_pidm order by sgbstdn_term_code_eff desc) = 1
    ) as C
on
    sfrstcr_pidm = sgbstdn_pidm
where
    sfrstcr_term_code = {self.term_code}
    and sfrstcr_error_flag is null
    and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term part
    and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
    and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
    and ssbsect_subj_code <> 'INST' -- exceptional sections
    and ssbsect_credit_hrs > 0
group by
    {indent(join(dct.keys()))}
    ,ssbsect_subj_code
    ,ssbsect_crse_numb
"""

            qry = f"""
with A as {subqry(qry)}
select * from A

union all

select
    {indent(join(dct.values()))}
    ,'_allcrse' as crse_code
    ,sum(credit_hr) as credit_hr
from A
group by
    {indent(join(dct.values()))}

union all

select
    {indent(join(dct.values()))}
    ,'_anycrse' as crse_code
    ,1 as credit_hr
from A
group by
    {indent(join(dct.values()))}
"""
            df = run(qry, show).set_index(['crse_code','pidm'])
            return df
        df, new = self.get(fcn, 'registrations')
        return df


    def get_enrollments(self, show=False):
        return self.get_registrations().query(f"levl_code=='ug' & styp_code in ['n','t','r']").groupby(['crse_code','styp_code']).size().rename('headcount').to_frame()
##############################################
############### get admissions ###############
##############################################
    def get_admissions(self, show=False):
        def fcn():
            L = [f"""
select
    *
from (
    select distinct
        A.*
        ,min(current_date) over (partition by pidm, appl_no) as first_date
        ,max(current_date) over (partition by pidm, appl_no) as final_date
    from
        dev.opeir.opeiradmissions_{self.terms.loc[term_code,'term_desc']} as A
        --dev.opeir.admissions_{self.terms.loc[term_code,'term_desc']}_v as A
    inner join
        {catalog}saturnstvapdc as B
    on
        apdc_code = stvapdc_code
    where
        stvapdc_inst_acc_ind is not null  --only accepted
    qualify
        '{self.cycle_date}' between first_date and final_date --keep only pidm, appl_no where cycle_date falls between its first and last records
    )
where
    current_date <= '{self.cycle_date}'  -- only records before cycle_date
qualify
    row_number() over (partition by pidm, appl_no order by current_date desc) = 1  -- most current record remaiming for this pidm, appl_no
"""
                for term_code in [self.term_code-2, self.term_code]]
            qry = join(L, '\nunion all\n')

            def get_desc(code):
                for nm in code.split('_'):
                    if len(nm) == 4:
                        break
                return [f'{code} as {nm}_code, stv{nm}_desc as {nm}_desc', f'left join {catalog}saturnstv{nm} on {code} = stv{nm}_code']
            
            def coalesce(x,y=False):
                return f'coalesce({x}, {y}) as {x}'

            qry = f"""
select
    pidm
    ,{self.cycle_day} as cycle_day
    ,timestamp('{self.cycle_date}') as cycle_date
    ,current_date
    ,first_date
    ,final_date
    ,{get_desc('term_code')[0]}
    ,appl_no
    ,{get_desc('apst_code')[0]}
    ,{get_desc('apdc_code')[0]}
    ,{get_desc('admt_code')[0]}
    ,{get_desc('wrsn_code')[0]}
    ,{get_desc('levl_code')[0]}
    ,{get_desc('styp_code')[0]}    
    ,{get_desc('camp_code')[0]}
    ,{get_desc('coll_code_1')[0]}
    ,{get_desc('dept_code')[0]}
    ,{get_desc('majr_code_1')[0]}
    ,{get_desc('saradap_resd_code')[0]}
    ,gender
    ,birth_date
    ,{get_desc('spbpers_lgcy_code')[0]}
    ,gorvisa_vtyp_code is not null as international
    ,gorvisa_natn_code_issue as natn_code, stvnatn_nation as natn_desc
    ,{coalesce('race_asian')}
    ,{coalesce('race_black')}
    ,coalesce(spbpers_ethn_cde=2, False) as race_hispanic
    ,{coalesce('race_native')}
    ,{coalesce('race_pacific')}
    ,{coalesce('race_white')}
    ,hs_percentile
    ,sbgi_code
    ,enrolled_ind='Y' as enrolled_ind

from {subqry(qry)} as A

left join
    {catalog}spbpers_amp_v
on
    pidm = spbpers_pidm

left join (
    select
        *
    from
        {catalog}generalgorvisa
        --{catalog}gorvisa_amp_v
    qualify
        row_number() over (partition by gorvisa_pidm order by gorvisa_seq_no desc) = 1
    )
on
    pidm = gorvisa_pidm

left join (
    select
        gorprac_pidm
        ,max(gorprac_race_cde='AS') as race_asian
        ,max(gorprac_race_cde='BL') as race_black
        ,max(gorprac_race_cde='IN') as race_native
        ,max(gorprac_race_cde='HA') as race_pacific
        ,max(gorprac_race_cde='WH') as race_white
    from
        {catalog}generalgorprac
    group by
        gorprac_pidm
    )
on
    pidm = gorprac_pidm

{get_desc('term_code')[1]}
{get_desc('levl_code')[1]}
{get_desc('styp_code')[1]}
{get_desc('admt_code')[1]}
{get_desc('wrsn_code')[1]}
{get_desc('apst_code')[1]}
{get_desc('apdc_code')[1]}
{get_desc('camp_code')[1]}
{get_desc('coll_code_1')[1]}
{get_desc('dept_code')[1]}
{get_desc('majr_code_1')[1]}
{get_desc('saradap_resd_code')[1]}
{get_desc('gorvisa_natn_code_issue')[1]}
{get_desc('spbpers_lgcy_code')[1]}

qualify
    min(levl_code='UG') over (partition by pidm) = True  -- remove pidm's with graduate admission even it if also has an undergradute admission, min acts like logical and
    and row_number() over (partition by pidm order by appl_no desc) = 1  -- de-duplicate the few remaining pidms with multiple record by keeping highest appl_no
"""
            df = run(qry, show)
            return df
        df, new = self.get(fcn, 'admissions')
        return df


    def get_students(self, show=False):
        def fcn():
            df = (self.admissions
                  .merge(self.flags, on=['pidm','term_code'], how='left', suffixes=['', '_flags'])
                  .merge(self.drivetimes, on=['zip','camp_code'], how='left', suffixes=['', '_zips'])                
            )
            for c in ['gap_score','t_gap_score','ftic_gap_score']:
                if c not in df:
                    df[c] = pd.NA
            
            df['gap_score'] = np.where(
                df['styp_code']=='n',
                df['ftic_gap_score'].combine_first(df['t_gap_score']).combine_first(df['gap_score']),
                df['t_gap_score'].combine_first(df['ftic_gap_score']).combine_first(df['gap_score']))
            
            
            # df['oriented'] = np.where(df['orien_sess'].notnull() | df['registered'].notnull(), 'y', np.where(df['orientation_hold_exists'].notnull(), 'n', 'w'))
            df['oriented'] = df['orientation_hold_exists'].isnull() | df['orien_sess'].notnull() | df['registered'].notnull()

            # df['verified'] = np.where(df['ver_complete'].notnull(), 'y', np.where(df['selected_for_ver'].notnull(), 'n', 'w'))
            df['verified'] = df['selected_for_ver'].isnull() | df['ver_complete'].notnull()
            
            df['sat10_total_score'] = (36-9) / (1600-590) * (df['sat10_total_score']-590) + 9
            df['act_equiv'] = df[['act_new_comp_score','sat10_total_score']].max(axis=1)

            df['eager'] = (self.stable_date - df['first_date']).dt.days
            df['age'] = (self.stable_date - df['birth_date']).dt.days

            for k in ['reading', 'writing', 'math']:
                df[f'tsi_{k}'] = ~df[k].isin(['not college ready', 'retest required', pd.NA, None, np.nan])
            
            repl = {'ae':0, 'n1':1, 'n2':2, 'n3':3, 'n4':4, 'r1':1, 'r2':2, 'r3':3, 'r4':4}
            df['hs_qrtl'] = pd.cut(df['hs_pctl'], bins=[-1,25,50,75,90,101], labels=[4,3,2,1,0], right=False).combine_first(df['apdc_code'].map(repl))

            df['lgcy'] = ~df['lgcy_code'].isin(['o',pd.NA,None,np.nan])
            df['resd'] = df['resd_code'] == 'r'

            for k in ['waiver_desc','fafsa_app','ssb_last_accessed','finaid_accepted','schlship_app']:
                df[k.split('_')[0]] = df[k].notnull()



            # df['majr_code'] = df['majr_code'].replace({'0000':pd.NA, 'und':pd.NA, 'eled':'eted', 'agri':'unda'})

            # df['coll_code'] = df['coll_code'].replace({'ae':'an', 'eh':'ed', 'hs':'hl', 'st':'sm', '00':pd.NA})

            # df['coll_desc'] = df['coll_code'].map({
            #     'an': 'ag & natural_resources',
            #     'ba': 'business',
            #     'ed': 'education',
            #     'en': 'engineering',
            #     'hl': 'health sciences',
            #     'la': 'liberal & fine arts',
            #     'sm': 'science & mathematics',
            #     pd.NA: 'no college designated',
            # })



            # checks = [
            #     'cycle_day >= 0',
            #     'eager >= cycle_day',
            #     'age >= 5000',
            #     'distance >= 0',
            #     'hs_pctl >=0',
            #     'hs_pctl <= 100',
            #     'hs_qrtl >= 0',
            #     'hs_qrtl <= 4',
            #     'act_equiv >= 1',
            #     'act_equiv <= 36',
            #     'gap_score >= 0',
            #     'gap_score <= 100',
            # ]
            # for check in checks:
            #     mask = df.eval(check)
            #     assert mask.all(), [check,df[~mask].disp(5)]

            return df.set_index(['pidm'])
        
        df, new = self.get(fcn, 'students', [
            self.get_admissions,
            self.get_flags,
            self.get_drivetimes,
            ])
        mask = df['cycle_date_flags'].isnull()  # rows from admissions not on flags - should not be any
        if mask.any():
            display(df[mask]['styp_code'].value_counts().sort_index().to_frame().T)
        return df

####################################
############### main ###############
####################################
@dataclasses.dataclass
class AMP(Term):
    crse_code : str = '_anycrse'

    def __post_init__(self):
        super().__post_init__()

self = AMP(
    overwrite=[
        'drivetimes',
    ]
)
self.get_drivetimes()

In [0]:
crse_codes = [
    '_anycrse',
]

# date = '03-06'
date = '06-26'
for crse_code in crse_codes:
    def kwargs(year):
        return {
            'cycle_date': f'{year}-{date}',
            'term_code': 100*year+8,
            'crse_code': crse_code,
            'overwrite': {
                # 'terms',
                # 'zips',
                # 'drivetimes',
                # 'flags',
                # 'admissions',
                # 'students',
                # 'registrations',
                # 'enrollments',
                },
            }
    for year in [2024]:
        curr = AMP(**kwargs(year))
        stab = AMP(**kwargs(year), cycle_day=0)
        pred = AMP(**kwargs(2025))
        # curr.get_distances()
        # curr.process_flags()
        stab.get_registrations()
        for self in [curr, pred]:
            self.get_students()
            self.get_registrations()

            features = {
                # 'act_equiv':pd.NA,
                'age':pd.NA,
                'camp_desc':'stephenville',
                'eager':pd.NA,
                'fafsa': False,
                'finaid': False,
                'gap_score':0,
                'gender':pd.NA,
                'hs_qrtl':pd.NA,
                'lgcy':False,
                'oriented':False,
                'race_asian':False,
                'race_black':False,
                'race_hispanic':False,
                'race_native':False,
                'race_pacific':False,
                'race_white':False,
                'schlship': False,
                'ssb': False,
                'tsi_math':False,
                'tsi_reading':False,
                'tsi_writing':False,
                'verified':False,
                'waiver': False,
                'credit_hr':0,
                'current':False,
            }

            df = (
                self.students
                .join(self.registrations.loc['_allcrse','credit_hr'])
                .join(self.registrations.loc[crse_code ,'credit_hr'].rename('current')>0)
                .join(stab.registrations.loc[crse_code ,'credit_hr'].rename('stable' )>0)
                .query("levl_code=='ug' & styp_code in ['n','r','t']")
                .fillna(features)
                .prep()
            )
            for k,v in df.select_dtypes('string').items():
                df[k] = pd.Categorical(v)    

            self.X = dict()
            self.y = dict()
            self.Z = dict()
            for s, Z in df.groupby('styp_code', observed=True):
                y = Z.pop('stable').fillna(False)
                imp = mf.ImputationKernel(Z[features.keys()].reset_index(drop=True), random_state=seed)
                imp.mice(10)
                X = imp.complete_data().set_index(Z.index)

                if self == curr:
                    dct = {
                        'time_budget':10,
                        'task':'classification',
                        'verbose':0,
                        'metric':'f1',
                        'eval_method':'cv',
                        'n_splits':3,
                        'seed':seed,
                        # 'early_stop':True,
                        'estimator_list': ['xgboost'],
                    }
                    clf = fl.AutoML(**dct)
                    clf.fit(X, y, **dct)

                self.Z[s] = X.assign(proba=clf.predict_proba(X)[:,1], pred=clf.predict(X), true=y)
                self.clf[s] = clf
            break

                # c = ['levl_code','styp_code']
        
        #         curr.registrations = curr.registrations.drop(columns=c).join(self.students[c])
        #         curr.enrollments = (
        #             curr.get_enrollments()
        #             .join(stab.get_enrollments(), how='outer', rsuffix='_')
        #             .set_axis(['current','stable'], axis=1)
        #             .fillna(0)
        #             .assign(mlt=lambda X: X['stable'] / X['current'])
        #             .fillna(0)
        # )

In [0]:
## useful old code
#     def get_registrations(self, overwrite=False, show=False):
#         def fcn():
#             dct = {
#                 'sfrstcr_pidm':'pidm',
#                 'ssbsect_term_code':'term_code',
#                 'sgbstdn_levl_code':'levl_code',
#                 'sgbstdn_styp_code':'styp_code',
#                 'ssbsect_crn':'crn',
#             }
#             qry = f"""
# select
#     {indent(join(alias(dct)))}
#     ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
#     ,max(ssbsect_credit_hrs) as credit_hr
# from
#     {catalog}saturnsfrstcr as A
# inner join
#     {catalog}saturnssbsect as B
# on
#     sfrstcr_term_code = ssbsect_term_code
#     and sfrstcr_crn = ssbsect_crn
# inner join (
#     select
#         *
#     from
#         {catalog}sgbstdn_amp_v
#     where
#         sgbstdn_term_code_eff <= {self.term_code}
#     qualify
#         row_number() over (partition by sgbstdn_pidm order by sgbstdn_term_code_eff desc) = 1
#     ) as C
# on
#     sfrstcr_pidm = sgbstdn_pidm
# where
#     sfrstcr_term_code = {self.term_code}
#     and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term part
#     and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
#     and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
#     and ssbsect_subj_code <> 'INST' -- exceptional sections
# group by
#     {indent(join(dct.keys()))}
#     ,ssbsect_subj_code
#     ,ssbsect_crse_numb
# """

#             qry = f"""
# with A as {subqry(qry)}
# select * from A

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_allcrse' as crse_code
#     ,sum(credit_hr) as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_anycrse' as crse_code
#     ,case when sum(credit_hr) > 0 then 1 else 0 end as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}
# """
#             df = run(qry, show)
#             return df
#         df, new = self.get(fcn, 'registrations', overwrite)
#         return df


#     def get_registrations(self, overwrite=False, show=False):
#         def fcn():
#             dct = {
#                 'sfrstcr_pidm':'pidm',
#                 'ssbsect_term_code':'term_code',
#             }
#             qry = f"""
# select
#     {indent(join(alias(dct)))}
#     ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
#     ,max(ssbsect_credit_hrs) as credit_hr
# from
#     {catalog}saturnsfrstcr as A
# inner join
#     {catalog}saturnssbsect as B
# on
#     sfrstcr_term_code = ssbsect_term_code
#     and sfrstcr_crn = ssbsect_crn
# where
#     sfrstcr_term_code = {self.term_code}
#     and sfrstcr_error_flag is null
#     and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term part
#     and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
#     and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
#     and ssbsect_subj_code <> 'INST' -- exceptional sections
# group by
#     {indent(join(dct.keys()))}
#     ,ssbsect_subj_code
#     ,ssbsect_crse_numb
# """

#             qry = f"""
# with A as {subqry(qry)}
# select * from A

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_allcrse' as crse_code
#     ,sum(credit_hr) as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_anycrse' as crse_code
#     ,case when sum(credit_hr) > 0 then 1 else 0 end as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}
# """
#             df = run(qry, show)
#             return df
#         df, new = self.get(fcn, 'registrations', overwrite)

#         S = self.students[['pidm','term_code','styp_code']]
#         return df



    #     def fcn():
#             qry = f"""
# select
#     sfrstcr_pidm as pidm
#     ,sfrstcr_term_code as term_code
#     ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
#     ,max(ssbsect_credit_hrs) as credit_hr
# from
#     {catalog}saturnsfrstcr as A
# inner join
#     {catalog}saturnssbsect as B
# on
#     sfrstcr_term_code = ssbsect_term_code
#     and sfrstcr_crn = ssbsect_crn
# where
#     sfrstcr_term_code = {self.term_code}
#     --and sfrstcr_error_flag is null
#     and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term part
#     and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
#     and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
#     and ssbsect_subj_code <> 'INST' -- exceptional sections
# group by
#     sfrstcr_pidm
#     ,sfrstcr_term_code
#     ,ssbsect_subj_code
#     ,ssbsect_crse_numb
# """

#             qry = f"""
# with A as {subqry(qry)}
# select * from A

# union all

# select
#     pidm
#     ,term_code
#     ,'_allcrse' as crse_code
#     ,sum(credit_hr) as credit_hr
# from A
# group by
#     pidm
#     ,term_code

# union all

# select
#     pidm
#     ,term_code
#     ,'_anycrse' as crse_code
#     ,case when sum(credit_hr) > 0 then 1 else 0 end as credit_hr
# from A
# group by
#     pidm
#     ,term_code
# """


# don't delete - could be useful & was hard to create
            # stat_codes = ['AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VA','VT','WA','WI','WV','WY'] # not AK & HI b/c can't get driving distance
#     ,{get_desc('spraddr_cnty_code')[0]}
#     ,{get_desc('spraddr_stat_code')[0]}
#     ,zip_code

# left join (
#     select
#         *
#         ,try_to_number(left(spraddr_zip, 5), '00000') as zip_code
#         ,case
#             when spraddr_atyp_code = 'PA' then 6
#             when spraddr_atyp_code = 'PR' then 5
#             when spraddr_atyp_code = 'MA' then 4
#             when spraddr_atyp_code = 'BU' then 3
#             when spraddr_atyp_code = 'BI' then 2
#             when spraddr_atyp_code = 'P1' then 1
#             when spraddr_atyp_code = 'P2' then 0
#             end as spraddr_atyp_rank
#     from
#         {catalog}spraddr_amp_v
#     where
#         spraddr_stat_code in ('{join(stat_codes, "','")}')
#         and spraddr_zip is not null
#     qualify
#         row_number() over (partition by spraddr_pidm order by spraddr_atyp_rank desc, spraddr_seqno desc) = 1
# )
# on
#     pidm = spraddr_pidm

# {get_desc('spraddr_cnty_code')[1]}
# {get_desc('spraddr_stat_code')[1]}



    # def get_zips(self, show=False):
    #     """takes ~3 hours toget zip codes and find nearest point on road network to the provided representative point"""
    #     def fcn():
    #         from pgeocode import Nominatim
    #         nomi = Nominatim('us')
    #         df = nomi.query_postal_code(pd.Series(nomi._data['postal_code'])).query("state_code.notnull() & state_code not in ['AK', 'HI', 'MH']").prep().set_index('postal_code').rename_axis('zip')
    #         nearest = lambda x: join(requests.get(f"http://router.project-osrm.org/nearest/v1/driving/{x['longitude']},{x['latitude']}").json()['waypoints'][0]['location'],',')
    #         df['point'] = df.apply(nearest, axis=1)
    #         return df
    #     df, new = self.get(fcn, root/'zips')
    #     self.states = set(df['state_code'])
    #     return df


    # def get_drivetimes(self, show=False):
    #     def fcn():
    #         campus_coords = {
    #             's': [-98.215784,32.216217],
    #             # 'm': '-97.432975,32.582436',
    #             # 'w': 76708,
    #             # 'r': 77807,
    #             }

    #         url = "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_zcta520_500k.zip"
    #         gdf = gpd.read_file(url).prep().set_index('zcta5ce20').iloc[:5]
    #         pts = gdf.sample_points(size=5,method="uniform").explode()#.apply(lambda geom: f"{geom.x},{geom.y}")
    #         df = pts.to_frame()[[]]
    #         url = "http://router.project-osrm.org/table/v1/driving"
    #         headers = {"Content-Type": "application/json"}
    #         for k, v in campus_coords.items():
    #             u = [v, *pts]
    #             print(u)
    #             data = {
    #                 "coordinates": u,
    #                 "annotations": ["duration", "distance"],
    #                 "sources": 0,
    #             }
    #             response = requests.post(url, json=data, headers=headers)
    #             print(response.json())
    #             assert 1==2

            # for k, v in campus_coords.items():
            #     u = join([v, *pts], ';')
            #     url = f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={0}&annotations=duration,distance"
            #     print(url)
            #     print(requests.get(url))
            #     df[k] = np.squeeze(requests.get(url).json()['durations'])[1:]/60
    #         # df.disp(10)
    #         # df = df.groupby('zip').min()
    #         # df.disp(10)
    #         df = df.groupby(level=0).min().stack().reset_index().set_axis(['zip','camp_code','drivetime'], axis=1)
    #         return df

            # df = self.zips.iloc[34339:34349].copy()
            # u = join(df.apply(lambda x: f"{x['longitude']},{x['latitude']}", axis=1),';')
            # for k, z in campus_zips.items():
            #     df[k] = np.squeeze(
            #     requests.get(f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={df.index.get_loc(z)}&annotations=duration,distance&fallback_speed=26.8&fallback_coordinate=snapped"
            #     ).json()['distances'])/1609

            # for k, z in campus_zips.items():
            #     df[k] = np.squeeze(
            #     requests.get(f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={df.index.get_loc(z)}&annotations=duration,distance&fallback_speed=26.8&fallback_coordinate=snapped"
            #     ).json()['durations'])/60

            # self.zips = self.zips.iloc[34339:34349]
            # self.zips.disp(20)
            # u = join(self.zips['point'],';')
            
            # dct = dict()
            # for k, z in campus_zips.items():
            #     i = self.zips.index.get_loc(z)
            #     print(self.zips.iloc[i])
            #     url = f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={0}&annotations=duration,distance&fallback_speed=26.8&fallback_coordinate=snapped"
            #     print(url)
            #     response = requests.get(url).json()
            #     for k,v in response.items():
            #         print(k)
            #         display(v)
            #         print()
            #     print(response['distance'])
            #     dct[k] = np.squeeze(response['durations'])

            # # dct = {k: np.squeeze(
            # #     requests.get(f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={self.zips.index.get_loc(z)}&fallback_speed=600&fallback_coordinate=snapped"
            # #     ).json()['durations'])/60 for k, z in campus_zips.items()}
            # dct = {k: np.squeeze(
            #     requests.get(f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={self.zips.index.get_loc(z)}&annotations=duration,distance&fallback_speed=26.8&fallback_coordinate=snapped"
            #     ).json()['distances']) for k, z in campus_zips.items()}

            # df = pd.DataFrame(dct, index=self.zips.index).stack().rename_axis(['zip','camp_code']).rename('drivetime') / 1609
            # return df
            # print()
            # dct = {k: self.zips.loc[y] for k, y in {
            #     's': 76402,
            #     'm': 76036,
            #     'w': 76708,
            #     'r': 77807,
            #     }.items()}
            # L = [
            #     self.get(
            #         lambda: X.apply(get_driving_distance, y=y, axis=1).rename('distance').reset_index().assign(camp_code=k),
            #         root/f'distances/distances_{s}_{k}',
            #         divide=False,
            #     )[0] for s, X in self.zips.groupby('state_code') for k, y in dct.items()]
            # return pd.concat(L, ignore_index=True)
        # df, new = self.get(fcn, root/'drivetimes')#, self.get_zips)
        # return df




    # def get_flags_history(self, cutoff=202206):
    #     def fcn():
    #         import pyarrow.parquet as pq
    #         print()
    #         L = []
    #         for path in sorted(flags_prc.iterdir(), reverse=True):
    #             print(path)
    #             for src in path.iterdir():
    #                 _, term_code, cycle_date, cycle_day = src.stem.split('_')
    #                 col = pq.ParquetFile(src).schema.names
    #                 df = pd.DataFrame(columns=col).assign(term_code=[int(term_code)], cycle_date=[cycle_date]).fillna(True)
    #                 L.append(df)
    #         df = pd.concat(L).fillna(False).set_index(['cycle_date','term_code']).sort_index()
    #         return df[sorted(df.columns)]
    #     df, new = self.get(fcn, path=data/'flags_history')
    #     A = df.query(f'term_code>={cutoff}').groupby('term_code').sum().sort_index(ascending=False).T.rename_axis('variable')
    #     B = A == A.max()
    #     B.insert(0, 'n', B.sum(axis=1))
    #     return B.reset_index().sort_values(['n', 'variable'], ascending=[False, True])



############ annoying warnings to suppress ############
# [warnings.filterwarnings(action='ignore', message=f".*{w}.*") for w in [
#     "Could not infer format, so each element will be parsed individually, falling back to `dateutil`",
#     "Engine has switched to 'python' because numexpr does not support extension array dtypes",
#     "The default of observed=False is deprecated and will be changed to True in a future version of pandas",
#     "errors='ignore' is deprecated"
#     "The behavior of DataFrame concatenation with empty or all-NA entries is deprecated",
#     "The behavior of array concatenation with empty entries is deprecated",
#     "DataFrame is highly fragmented",
# ]]

